#**EXERCISE-2**

In [ ]:
import numpy as np
import timeit
np.random.seed(1000)

In [ ]:
import math

In [ ]:
def evalf(x,lamda, A,y):
  assert type(x) is np.ndarray 
  return (lamda/2)*np.matmul(x.T,x)+0.5*(np.linalg.norm(np.matmul(A,x) - y))**2

In [ ]:
def evalg(x, lamda, A,y):
  assert type(x) is np.ndarray
  return np.add(np.matmul(A.T, np.subtract(np.matmul(A, x), y)), lamda * x)

In [ ]:
def evalh(x, lamda, A,y):
  assert type(x) is np.ndarray
  return np.add(np.matmul(A.T,A), lamda * np.identity(len(x)))

In [ ]:
BACKTRACKING_LINE_SEARCH = 1

In [ ]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_start, rho, gamma,lamda,A,y):
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray 
  assert type(direction) is np.ndarray 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  alpha = alpha_start
  while evalf(x+alpha*direction, lamda, A,y) > (evalf(x,lamda,A,y) + gamma*alpha*np.matmul(gradf.T,direction)):
    alpha=rho*alpha

  return alpha

In [ ]:
def find_minimizer_Newton(A, y, start_x, tol, line_search_type,lamda, *args):
  assert type(start_x) is np.ndarray
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,lamda,A,y)

  alpha_start = float(args[0])
  rho = float(args[1])
  gamma = float(args[2])

  k = 0
  while (np.linalg.norm(g_x) > tol):
    D_k = np.linalg.inv(evalh(x,lamda, A,y))
    p_k = -np.matmul(D_k,g_x)
    step_length = compute_steplength_backtracking_scaled_direction(x, g_x,p_k, alpha_start, rho, gamma,lamda,A,y) 

    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1
    g_x = evalg(x,lamda, A,y) 

  return x,  k, evalf(x,lamda,A,y)

In [ ]:
def find_minimizer_BFGS(A,y, start_x, tol, line_search_type, lamda, *args):
  assert type(start_x) is np.ndarray 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x, lamda,A,y)
  I = np.identity(len(x))
  B_k = I

  alpha_start = float(args[0])
  rho = float(args[1])
  gamma = float(args[2])

  k = 0
  while (np.linalg.norm(g_x) > tol): 
    p_k = -np.matmul(B_k, g_x)
    step_length = compute_steplength_backtracking_scaled_direction(x, g_x,p_k, alpha_start, rho, gamma,lamda,A,y)

    x_prev = x
    s_k = np.multiply(step_length,p_k) 
    x = np.add(x, s_k)
    y_k = evalg(x,lamda, A,y)-evalg(x_prev,lamda, A,y)

    u_k = 1/(np.matmul(y_k.T,s_k))
    term_1 = np.subtract(I , u_k*np.matmul(s_k,y_k.T))
    term_2 = np.subtract(I , u_k*np.matmul(y_k, s_k.T))
    B_k = np.matmul(np.matmul(term_1,B_k), term_2) + u_k*np.matmul(s_k,s_k.T)

    k += 1 #increment iteration
    g_x = evalg(x,lamda, A,y) #compute gradient at new point

  return x , k, evalf(x,lamda, A,y)

In [ ]:
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lamda_reg = 0.001
eps = np.random.randn(N,1)

for  i in range(np.size(ds)):
  d = ds[i]
  A = np.random.randn(N,d)
  for j in range(A.shape[1]):
    A[:,j] = A[:,j]/np.linalg.norm(A[:,j])

  xorig = np.ones((d,1))
  y = np.dot(A, xorig) + eps
  n = len(xorig)
  my_tol= 1e-3
  lamda = 0.001
  start  = timeit.default_timer()
  x, k, f_value = find_minimizer_Newton(A,y, xorig , my_tol, BACKTRACKING_LINE_SEARCH,lamda, 0.9 , 0.5, 0.5)
  newtontime = timeit.default_timer() - start
  
  start_bfgs  = timeit.default_timer()
  x_bfgs, k_bfgs,f_value_bfgs = find_minimizer_BFGS(A,y, xorig , my_tol, BACKTRACKING_LINE_SEARCH,lamda, 0.9 , 0.5, 0.5)
  bfgstime = timeit.default_timer() - start_bfgs
   
  print("\nFor d = ", d)
  print("For Newton: ")
  print("time: ", newtontime )
  print("||Ax_opt - y||^2 :", (np.linalg.norm(np.matmul(A,x) - y))**2)
  print(" ||x_opt-xorig||^2 :", (np.linalg.norm(x - xorig))**2)
  print("For BFGS: ")
  print("time: ", bfgstime )
  print("||Ax_opt - y||^2 :", (np.linalg.norm(np.matmul(A,x_bfgs) - y))**2)
  print(" ||x_opt-xorig||^2 :", (np.linalg.norm(x_bfgs - xorig))**2)



For d =  1000
For Newton: 
time:  1.1063840599999821
||Ax_opt - y||^2 : 5.77140010314599e-05
 ||x_opt-xorig||^2 : 865.2979971977057
For BFGS: 
time:  7.343680744000267
||Ax_opt - y||^2 : 5.571737379621836e-05
 ||x_opt-xorig||^2 : 865.3523026794429

For d =  5000
For Newton: 
time:  54.6139351769998
||Ax_opt - y||^2 : 9.949803059501234e-06
 ||x_opt-xorig||^2 : 4783.586019993549
For BFGS: 
time:  654.59976018
||Ax_opt - y||^2 : 9.7776996902963e-06
 ||x_opt-xorig||^2 : 4783.669438585895


#For Newton method session crashed at d = 10000

#For BFGS method session crashed at d = 10000 .